In [ ]:
!pip install boto3

In [1]:
# Import LUSID
import lusid
import lusid.models as models
import lusid_sample_data as import_data
import globalfund as global_fund_tools
import cocoon as cocoon_tools
# Import Libraries
import pprint
from datetime import datetime, timedelta, time, date
import pytz
import uuid
import printer as prettyprint
from datetime import datetime
import pandas as pd
import numpy as np
import os
import json
globals = {}

# Authenticate our user and create our API client
lusid_client = import_data.authenticate_secrets()

print('LUSID Client Initialised')
print('LUSID version : ', lusid_client.metadata.get_lusid_versions().build_version)

LUSID Client Initialised
LUSID version :  0.5.3213.0


## 1) Load Instrument Master across Regions

### Connect to AWS Athena

In [3]:
s3_client, athena_client = global_fund_tools.connect_athena_s3_clients(
    AWS_SERVER_PUBLIC_KEY="",
    AWS_SERVER_SECRET_KEY="",
    region='eu-west-2')

### Fetch the Instrument Master Data

#### US Instrument Master

In [2]:
query = """
select * from us_instrument_master
"""

us_instrument_master = global_fund_tools.fetch_data(
    athena_client=athena_client, 
    s3_client=s3_client, 
    database_name="instrument_master", 
    sql_query=query, 
    results_bucket="fbn-demo-query-results", 
    output_file_path="./data/instrument_master.csv", 
    retry_time=10)

us_instrument_master.head(n=20)

NameError: name 'athena_client' is not defined

#### UK Instrument Master

In [ ]:
query = """
select * from uk_instrument_master
"""

uk_instrument_master = global_fund_tools.fetch_data(
    athena_client=athena_client, 
    s3_client=s3_client, 
    database_name="instrument_master", 
    sql_query=query, 
    results_bucket="fbn-demo-query-results", 
    output_file_path="./data/instrument_master.csv", 
    retry_time=10)

uk_instrument_master.head(n=20)

#### Combined Instrument Master

In [ ]:
query = """
SELECT 
instrument_name, 
client_internal, 
currency, 
isin, 
figi, 
coupon as couprate, 
"s&p rating", 
null as moodys_rating FROM uk_instrument_master 

UNION ALL

SELECT 
name as instrument_name, 
int_id as client_internal,
isocur as currency, 
isin, 
figi, 
couprate, 
null as "s&p rating",  
moodys_rating FROM us_instrument_master
"""

combined_instrument_master = global_fund_tools.fetch_data(
    athena_client=athena_client, 
    s3_client=s3_client, 
    database_name="instrument_master", 
    sql_query=query, 
    results_bucket="fbn-demo-query-results", 
    output_file_path="./data/instrument_master.csv", 
    retry_time=10)

combined_instrument_master.head(n=30)

### Load the Instruments into LUSID

In [ ]:
instrument_identifier_mapping = {
    'identifier_mapping': {
        "Figi": "figi",
        "Isin": "isin",
        "ClientInternal": "client_internal"
    }
}

instrument_mapping_required = {
    "name": "instrument_name"
}

instrument_mapping_optional = {}

cocoon_tools.load_file_multiple_portfolios(
    client=lusid_client, 
    scope='InstrumentProperties0003', 
    data_frame=combined_instrument_master, 
    mapping_required=instrument_mapping_required, 
    mapping_optional=instrument_mapping_optional, 
    source=None, 
    file_type="instrument", 
    identifier_mapping=instrument_identifier_mapping, 
    property_columns=["s&p rating", "moodys_rating", "currency"])

## 2) Load Transactions

### Create Transaction Portfolios

In [ ]:
global_fund_tools.create_portfolios(
    client=lusid_client, 
    scopes=['UK_Thinkfolio', 'US_SimcorpDimension'], 
    code="GlobalCreditFund", 
    currency="EUR")

### Fetch the Transaction Data

#### US

In [ ]:
query = """
SELECT * from us_ibor_transactions WHERE portfolio_code='GlobalCreditFund'
"""

us_transactions = global_fund_tools.fetch_data(
    athena_client=athena_client, 
    s3_client=s3_client, 
    database_name="ibor_transactions", 
    sql_query=query, 
    results_bucket="fbn-demo-query-results", 
    output_file_path="./data/transactions.csv", 
    retry_time=10)

us_transactions.head(n=20)

#### UK

In [ ]:
query = """
SELECT * from uk_ibor_transactions WHERE fund_code='GlobalCreditFund'
"""

uk_transactions = global_fund_tools.fetch_data(
    athena_client=athena_client, 
    s3_client=s3_client, 
    database_name="ibor_transactions", 
    sql_query=query, 
    results_bucket="fbn-demo-query-results", 
    output_file_path="./data/transactions.csv", 
    retry_time=10)

uk_transactions.head(n=20)

#### Combined

In [4]:
query = """
SELECT 
fund_code as portfolio_code, 
t_id as id, 
"desc" as transaction_type,
trade_datetime as transaction_date,
settlement_datetime as settlement_date,
qty as units,
unit_price as transaction_price,
total_trade_amount as amount,
trade_currency,
null as instrument_name,
null as accounting_method,
null as mtom,
executor as broker_executor,
location as location_region,
isin_identifier as isin,
figi_identifier as figi,
internal_identifier as client_internal,
exposure as exposure_counterparty,
val,
compls,
'US_SimcorpDimension' as source

FROM uk_ibor_transactions WHERE fund_code='GlobalCreditFund'

UNION ALL

SELECT 
portfolio_code,
transaction_id as id,
transaction_type,
transaction_date,
settlement_date,
transaction_units as units,
transaction_price,
transaction_cost as amont,
transaction_currency as currency,
instrument_name,
accmethod as accounting_method,
mtom,
broker as broker_executor,
region as location_region,
figi,
isin,
client_internal,
company_exposure as exposure_counterparty,
null as val,
null as compls,
'UK_Thinkfolio' as source

FROM us_ibor_transactions WHERE portfolio_code='GlobalCreditFund'
"""

combined_transactions = global_fund_tools.fetch_data(
    athena_client=athena_client, 
    s3_client=s3_client, 
    database_name="ibor_transactions", 
    sql_query=query, 
    results_bucket="fbn-demo-query-results", 
    output_file_path="./data/transactions.csv", 
    retry_time=10)


for transaction_type in ["StartingBalance", "FundsIn"]:
    combined_transactions.loc[
        combined_transactions[
            "transaction_type"] == transaction_type, 'currency_transaction'] = combined_transactions["trade_currency"]

    
combined_transactions["exchange_rate"] = 1


combined_transactions.head(n=30)

,portfolio_code,id,transaction_type,transaction_date,settlement_date,units,transaction_price,amount,trade_currency,instrument_name,...,location_region,isin,figi,client_internal,exposure_counterparty,val,compls,source,currency_transaction,exchange_rate
0,GlobalCreditFund,kjk99898990z88908,StartingBalance,2019-09-01T09:31:22.664000+00:00,2019-09-01T09:31:22.664000+00:00,2500000000.00,1.00,2500000000.00,GBP,NaN,...,"Dublin, Ireland",NaN,NaN,NaN,NaN,532352352.00,MERC,US_SimcorpDimension,GBP,1
1,GlobalCreditFund,435opkplk03sfsf33,Purchase,2019-09-03T14:42:12.435000+00:00,2019-09-05T14:42:12.435000+00:00,250000.00,149.00,37250000.00,GBP,NaN,...,"Dublin, Ireland",GB00BN65R313,BBG006N6HZM7,imd_54234532,UKGovt,2335235380000.00,SFP,US_SimcorpDimension,NaN,1
2,GlobalCreditFund,dsfsdlkjs933234211,Purchase,2019-09-05T15:12:44.090000+00:00,2019-09-07T15:12:44.090000+00:00,1500000.00,104.00,156000000.00,GBP,NaN,...,"Dublin, Ireland",GB00B4RMG977,BBG001KKJLR4,imd_34643653,UKGovt,25252.00,LLK,US_SimcorpDimension,NaN,1
3,GlobalCreditFund,dfklsjd90232350239,Purchase,2019-09-10T09:58:43.112000+00:00,2019-09-11T09:58:43.112000+00:00,2200000.00,154.00,338800000.00,GBP,NaN,...,"Dublin, Ireland",GB00B52WS153,BBG0000D14P3,imd_34534534,UKGovt,222532.00,LLK,US_SimcorpDimension,NaN,1
4,GlobalCreditFund,kj0llk90f0909201141,Purchase,2019-09-10T13:58:57.113000+00:00,2019-09-11T13:58:57.113000+00:00,190000.00,108.00,20520000.00,GBP,NaN,...,"Luxembourg, Luxembourg",XS0414345974,NaN,imd_34001121,TESCO,235235232.00,MLLK,US_SimcorpDimension,NaN,1
5,GlobalCreditFund,ab1543673,FundsIn,2019-09-01T09:31:22.664000+00:00,2019-09-01T09:31:22.664000+00:00,1000000000.00,1.00,1000000000.00,USD,USD,...,CR,NaN,NaN,NaN,NaN,nan,NaN,UK_Thinkfolio,USD,1
6,GlobalCreditFund,xfg352235,BY,2019-09-03T18:42:12.435000+00:00,2019-09-05T18:42:12.435000+00:00,50000.00,99.00,4950000.00,USD,USTreasury_2.00_2021,...,CR,BBG00FN3B5K8,US912828U816,imd_34535347,USGovt,nan,NaN,UK_Thinkfolio,NaN,1
7,GlobalCreditFund,ccv335235,BY,2019-09-06T19:12:44.090000+00:00,2019-09-08T19:12:44.090000+00:00,130000.00,127.00,16510000.00,USD,USTreasury_6.875_2025,...,CR,BBG000DQQNJ8,US912810EV62,imd_34534539,USGovt,nan,NaN,UK_Thinkfolio,NaN,1
8,GlobalCreditFund,cvcg767j31,BY,2019-09-10T13:58:57.113000+00:00,2019-09-11T13:58:57.113000+00:00,1750000.00,99.00,173250000.00,USD,TESCO_6.15_2037,...,CR,NaN,US881575AC87,imd_54001121,TESCO,nan,NaN,UK_Thinkfolio,NaN,1


#### Load the Transactions into LUSID

In [7]:
transaction_field_mapping_required = {
    "portfolio_code": "portfolio_code",
    "transaction_id": "id",
    "transaction_type": "transaction_type",
    "transaction_date": "transaction_date",
    "settlement_date": "settlement_date",
    "units": "units",
    "transaction_price.price": "transaction_price",
    "total_consideration.amount": "amount",
    "total_consideration.currency": "trade_currency",
    "transaction_currency": "trade_currency"
    }

transaction_field_mapping_optional = {
    "exchange_rate": "exchange_rate"
}

transaction_identifier_mapping = {
    "identifier_mapping": {
        "Figi": "figi",
        "Isin": "isin",
        "ClientInternal": "client_internal"
    },
    "is_cash_with_currency": "currency_transaction"
}

combined_transactions = cocoon_tools.resolve_instruments(
    client=lusid_client,
    data_frame=combined_transactions,
    identifier_mapping=transaction_identifier_mapping)

for scope in ["UK_Thinkfolio", "US_SimcorpDimension"]:

    responses = cocoon_tools.load_file_multiple_portfolios(
        client=lusid_client, 
        scope=scope, 
        data_frame=combined_transactions.loc[
            combined_transactions["source"] == scope],
        mapping_required=transaction_field_mapping_required,
        mapping_optional=transaction_field_mapping_optional,
        source=scope,
        file_type='transaction',
        property_columns=[
            "instrument_name", 
            "accounting_method",
            "mtom",
            "broker_executor",
            "location_region",
            "exposure_counterparty",
            "val",
            "compls",
            ])

    for portfolio_code, response in responses.items():
        prettyprint.transactions_response(response, portfolio_code, scope)

Up to row 0
data types don't match for column val it is string in LUSID and float64 in file
Updated val to object
Check for missing transaction properties complete
Transactions Successfully Upserted into Portfolio
Scope: portfolio
Code: UK_Thinkfolio
Transactions Effective From: 2019-09-10 13:58:57.113000+00:00
Transactions Created On: 2019-09-11 18:22:13.691123+00:00

data types don't match for column accounting_method it is string in LUSID and float64 in file
Updated accounting_method to object
data types don't match for column mtom it is string in LUSID and float64 in file
Updated mtom to object
Check for missing transaction properties complete
Transactions Successfully Upserted into Portfolio
Scope: portfolio
Code: US_SimcorpDimension
Transactions Effective From: 2019-09-10 13:58:57.113000+00:00
Transactions Created On: 2019-09-11 18:22:15.523278+00:00



## 3) Configure Transaction Types

In [16]:
combined_transactions.groupby(["transaction_type", "source"]).agg({"units": "max"})

,,units
transaction_type,source,
BY,UK_Thinkfolio,1750000.00
FundsIn,UK_Thinkfolio,1000000000.00
Purchase,US_SimcorpDimension,2200000.00
StartingBalance,US_SimcorpDimension,2500000000.00


In [ ]:
global_fund_tools.create_transaction_type_configuration(
    client=lusid_client, 
    aliases=[
        ("BY", "Thinkfolio_abc"),
        ("Purchase", "SimcorpDimension_abc")
    ],
    movements=[
        models.TransactionConfigurationMovementDataRequest(
            movement_types='StockMovement',
            side='Side1',
            direction=1,
            properties=None,
            mappings=None),
        models.TransactionConfigurationMovementDataRequest(
            movement_types='CashCommitment',
            side='Side2',
            direction=-1,
            properties=None,
            mappings=None)
    ])


global_fund_tools.create_transaction_type_configuration(
    client=lusid_client, 
    aliases=[
        ("FundsIn", "Thinkfolio_abc"),
        ("StartingBalance", "SimcorpDimension_abc")
    ],
    movements=[
        models.TransactionConfigurationMovementDataRequest(
            movement_types='CashCommitment',
            side='Side2',
            direction=1,
            properties=None,
            mappings=None)
    ])

## 4) Create Cut Labels for Region Closes

In [ ]:
global_fund_tools.create_cut_labels(
    client=lusid_client,
    exchange_names=["LSE", "NYSE"], 
    cut_label_type="market_close")

## 5) Load Close of Day Prices

## 6) Group Portfolios

## 7) Value for each Region Close